In [1]:
%load_ext autoreload
%autoreload 2

## Image Matching Challenge (PhotoTourism)

In [2]:
from imc.imc_benchmark_utils import load_imc_results

scene_set = "test"  # or "val"
df = load_imc_results(f"imc/results/{scene_set}", return_df=True, all_scenes=False, auc_th=5)
print(len(df), "rows loaded")
# For each (method, custom_desc), get the row with the highest all_auc5, keeping all columns
idx = df.groupby(["method", "custom_desc"])["all_auc5"].idxmax()
df_grouped = df.loc[idx].reset_index(drop=True)
df_grouped = df_grouped.sort_values(by=["all_auc5"], ascending=False)

df_grouped.head(11)  # Show all 11 methods

2 rows loaded


,method,custom_desc,kpts_budget,params,all_rep,all_inliers,all_auc5
1,rdd,,2048kpts,matcher-mnn0.5-ratiotest0.98-ransac-0.75,41.6,323.0,42.2
0,disk,,2048kpts,matcher-mnn0.0-ratiotest0.95-ransac-0.75,45.3,390.0,38.7


In [ ]:
import math
import matplotlib.pyplot as plt

if scene_set == "val":
    metric = "all_auc5"
    df_plot = df.copy()
    df_plot = df_plot.sort_values(by=["method", "custom_desc", metric], ascending=[True, True, True])

    methods = df_plot['method'].unique()
    n_methods = len(methods)
    n_cols = 3
    n_rows = math.ceil(n_methods / n_cols)

    fig, axes = plt.subplots(n_rows, n_cols, figsize=(6 * n_cols, 4 * n_rows), squeeze=False)
    axes = axes.flatten()

    for i, method in enumerate(methods):
        ax = axes[i]
        df_method = df_plot[df_plot['method'] == method]
        # Group by custom_desc (e.g., '', 'SANDesc', 'G', etc.)
        for custom_desc, df_variant in df_method.groupby('custom_desc'):
            label = method if custom_desc == '' else f"{method} + {custom_desc}"
            x_vals = range(len(df_variant))
            y_vals = df_variant[metric].values
            marker = 'o' if custom_desc == '' else 's'
            linestyle = '-' if custom_desc == '' else '--'
            ax.plot(x_vals, y_vals, label=label, marker=marker, linestyle=linestyle, linewidth=2)
            for x, y, idx in zip(x_vals, y_vals, df_variant.index):
                ax.annotate(str(idx), (x, y), textcoords="offset points", xytext=(0,10), ha='center', fontsize=10)

        ax.set_title(method)
        ax.set_xlabel("")
        ax.set_xticks([])
        ax.set_ylabel(metric)
        ax.legend(loc='best')
        ax.grid(True, alpha=0.3)
        ax.set_ylim(df_plot[metric].min() - 1, df_plot[metric].max() + 5)

    # Hide unused subplots
    for j in range(i + 1, len(axes)):
        fig.delaxes(axes[j])

    plt.tight_layout()
    plt.suptitle(f"IMC Benchmark - {metric} on {scene_set} set", y=1.02, fontsize=16)
    plt.show()

## HPatches

In [4]:
from hpatches.hpatches_benchmark_utils import display_hpatches_results

print("===== HPatches Overall Results =====")
df = display_hpatches_results(partition='overall', ths=[1, 2, 3], tostring=False)  # Display results for thresholds 1, 2, and 3
df[[col for col in df.columns if "Rep" not in col]]

===== HPatches Overall Results =====


,Method,Custom Desc,MA@1,MA@2,MA@3,MS@1,MS@2,MS@3,HA@1,HA@2,HA@3
0,aliked,,40.4,64.3,74.6,26.3,40.9,46.9,39.3,65.0,77.6
1,aliked,SANDesc,45.0,72.3,83.0,26.7,41.1,46.5,43.7,65.0,77.6
2,dedode,,50.1,70.3,78.5,22.2,30.1,33.3,48.0,70.0,78.0
3,dedode,SANDesc,52.3,72.7,80.6,22.5,30.3,33.2,48.9,71.9,78.1
4,dedode-G,,48.4,68.6,77.0,22.4,30.8,34.2,48.0,69.8,79.1
5,disk,,45.6,67.5,76.7,27.7,39.8,44.7,39.8,61.7,72.8
6,disk,SANDesc,49.0,73.4,84.0,27.7,39.9,45.0,40.4,62.6,73.5
7,disk-kornia,,41.0,63.3,74.1,2732.2,5030.0,5940.4,46.1,65.2,75.0
8,disk-kornia,,41.0,63.3,74.1,2732.2,5030.0,5940.4,46.1,65.2,75.0
9,disk-kornia,,41.0,63.3,74.1,25.3,37.8,43.5,41.9,62.0,71.7


## Megadepth-1500

In [10]:
import os
import pandas as pd

results = 'megadepth1500/results/results.json'
if not os.path.exists(results):
    raise FileNotFoundError(f"Results file not found: {results}. First run the benchmark!")

df = pd.read_json(results).T.round(3)
df['inlier'] = round(df['inlier']).astype(int)
df['auc_5'] = [el*100 if el < 1 else el for el in df['auc_5']]
df['auc_10'] = [el*100 if el < 1 else el for el in df['auc_10']]
df['model'] = df.index.str.split(' ').str[0]
df['params'] = df.index.str.split(' ').str[1]
df['budget'] = df.index.str.split(' ').str[2]
df['note'] = df.index.str.split(' ').str[3]
df['timestamp'] = df.index.str.split(' ').str[-1]

# # filter
# df = df[
#     df['model'].str.contains('dedode') 
#     & ~df['note'].str.contains('None')
# ] # comment to display all budgets 

df = df[['model',  'budget', 'inlier', 'auc_5', 'auc_10', 'note', 'timestamp']]
# df.sort_values(by=['budget', 'model'], inplace=True, ascending=[True, True])
#  group by model, srot by auc5, keep first
df = df.sort_values(by=['model', 'auc_5'], ascending=[True, False])
df = df.groupby('model').first().reset_index()
df = df.sort_values(by=['auc_5'], ascending=False)
df.index = range(1, 1+len(df.index))
df

,model,budget,inlier,auc_5,auc_10,note,timestamp
1,dedode+SANDesc,2048,281,46.8,63.0,38911,20250913_225625
2,dedode-G,2048,297,45.9,63.3,None,20250913_153235
3,rdd,2048,402,45.6,62.7,20250923_141617,20250923_141617
4,dedode+SANDesc_paper,2048,273,45.3,61.5,None,20250913_154149
5,aliked,2048,501,39.7,55.9,20250919_232641,20250919_232641
6,dedode,2048,299,38.8,55.1,20250919_230343,20250919_230343
7,disk-kornia,2048,512,32.9,49.5,20250923_000159,20250923_000159
8,disk,2048,511,32.0,48.9,20250919_222933,20250919_222933
9,superpoint,2048,245,29.9,44.9,20250919_214215,20250919_214215


## Graz High Resolution

In [ ]:
scale_factors = {'scale_1.0': '4K', 'scale_1.5': 'QHD', 'scale_2.0': 'FHD'}

results = 'graz_high_res/results/results_paper.json'
if not os.path.exists(results):
    raise FileNotFoundError(f"Results file not found: {results}. First run the benchmark!")

df = pd.read_json(results).T.round(3)
df['inlier'] = round(df['inlier']).astype(int)
df['auc_5'] *= 100
df['auc_10'] *= 100
df['model'] = df.index.str.split(' ').str[0]
df['params'] = df.index.str.split(' ').str[1]
df['resolution'] = [scale_factors[s] for s in df.index.str.split(' ').str[2]]
df['budget'] = df.index.str.split(' ').str[3]
df['timestamp'] = df.index.str.split(' ').str[-1]

# filter
# df = df[df['resolution'].str.contains('FHD')] # comment to display all resolutions

df_auc = df[['model', 'budget', 'resolution', 'inlier', 'auc_5', 'auc_10', 'timestamp']].copy()
df_auc.sort_values(by=['resolution', 'model'], inplace=True, ascending=[False, True])
df_auc.index = range(1, 1+len(df_auc.index))
df_auc # there should be 11 elements for FHD, and 9 for QHD and 4K

### Scenes breakdown

Run this after running the benchmark, otherwise no statistics are available.

In [ ]:
import json
from benchmark_utils import pose_auc

with open('graz_high_res/results/results_paper.json', 'r') as f:
    data = json.load(f)
    
keys = list(data.keys())
auc_th = 5
scale = 1 # 1, 1.5, 2

breakdown, total = {}, {}
for key in keys:
    parts = key.split(' ')
    key = f'{parts[0]}_stats_scale_{float(scale)}_{parts[4]}_{parts[6]}'
    model = parts[0]

    try:
        os.path.exists(f'graz_high_res/stats/{key}.csv')
        df = pd.read_csv(f'graz_high_res/stats/{key}.csv')
    except FileNotFoundError:
        continue

    scenes = df.scene_name.unique()
    total[model] = round(pose_auc(df.e_pose, [auc_th])[0] * 100,1)

    for scene in scenes:
        scene_df = df[df.scene_name == scene]
        if scene not in breakdown:
            breakdown[scene] = {}
        breakdown[scene][model] = round(pose_auc(scene_df.e_pose, [auc_th])[0] * 100,1)

breakdown['total'] = total

In [ ]:
df_breakdown = pd.DataFrame.from_dict(breakdown, orient='index').T
df_breakdown